## Column Generation example

In [1]:
using HiGHS
using JuMP

In [2]:
mutable struct Label

    C :: Float64
    R :: Vector{Float64}
    s :: Integer
    V :: Vector{Float64}

    Label(C, R, V) = new(C, copy(R), sum(0 .< V), copy(V))

end

In [3]:
Label(L, n) = Label(Inf, zeros(Int, L), zeros(Int, n))

Label

In [4]:
import Base

In [5]:
"""
This function implements the idea of dominated labels, that is `x ≤ y` if and only if `y` is dominated by `x`
"""

function Base.:<=(x :: Label, y :: Label)

    return (x.C <= y.C) && all(x.R .<= y.R) && (x.s <= y.s) && all(p -> (p[1] == 0) || (p[2] > 0), zip(x.V, y.V))

end

In [6]:
function Base.isequal(x :: Label, y :: Label)

    # Attention to the use of == for real numbers!!
    return (x.C == y.C) && all(x.R .== y.R) && (x.s == y.s) && all(x.V .== y.V)

end

In [7]:
function Base.copy(λ :: Label)

    return Label(λ.C, λ.R, λ.V)

end

In [8]:
# Auxiliary functions
fromvec(v) = Label(v[1], [v[3]], v[4:end])
tovec(λ) = [λ.C; λ.s; λ.R...; λ.V...]

tovec (generic function with 1 method)

## ESPPRC

In [9]:
"""
Functionality: applies the label correcting algorithm on a totally connected graph for the ESPPRC.

### Input:
    V - set of vertices of the graph;
    S - matrix nxn where each entry represents the objective value gained by traveling along each edge of the graph;
    T - matrix nxn where T[i,j] corresponds to time used to travel from destination of i to origin of j + time from
    origin of j to destination of j;
    L - Float64 coresponding to upper limit for time resource;
    W - matrix nx2 containing the time window of each vertex;
    Lambda - matrix nx1 where each entry is the set of labels of the corresponding vertex;
    n - number of vertices of the graph.

### Output:
    Lambda - matrix nx1 where each entry is the set of labels of the corresponding vertex;
    bests - optimal objective function value (the entire label)

"""
function ESPPRC(V, S, T, L, W, Λ, n)
    
    bests = Label(1, n)
    
    i = 1
    
    j = 1
    
    E = zeros(n)

    E[1] = 1

    F = Set([])
    
    changed = false
    
    while sum(E) != 0
        
        if E[i] != 0
                
            for k in 0:(n-2) # i is the vertice we are treating and k varies along 0 to n-2
                
                j = (i+k) % n + 1  # j is a vertice we may extend a rotule to
                                
                for λ in Λ[i]

                    if λ.V[j] == 0 # if the extension is possible, we do that
                        
                        F = union(F, Set([Extend(λ, i, j, S, T, L, W)])) 

                    end
                    
                end
                
                Λ[j], changed, bests = EFF(Λ[j], F, bests)

                F = Set([])

                if changed
                    
                    E[j] = 1
                    
                end
                
            end
            
            E[i] = 0
            
        end
        
        i = i % n + 1
        
    end

    return Λ, bests

end

ESPPRC

## Extend

In [10]:
"""
Functionality: extend a label from vertice i to vertice j (by certification in ESPPRC(), the extension will always be possible)

### Input:
    lambda_i - matrix (2+n)x1 label to be extended from i;
    i - vertice that holds the current label;
    j - vertice that will receive a new label;
    T - matrix nxn where T[i,j] corresponds to time used to travel from destination of i to origin of j + time from
    origin of j to destination of j;
    L - Float64 coresponding to upper limit for time resource;
    W - matrix nx2 containing the time window of each vertex.

### Output:
    lambda_j - matrix (2+n)x1 (extended label).
"""
function Extend(λ_i, i, j, S, T, L, W)
    
    viavel = true

    λ_j = copy(λ_i)

    λ_j.R[1] = max(W[j, 1], λ_j.R[1] + T[i,j]) # we update time traveled and mark j as unreachable if lambda_j[2] + T[i,j] < W[j, 1] it means

    # that we arrived at the task before the beggining of the time window of excution of the task, o we must wait
    
    # uptading the order in which task j was done
    
    order = 1
    
    for k in λ_j.V
        if (k != Inf) && (k > order)
            order = k
        end
    end
    
    λ_j.V[j] = order + 1

    # updating resource usage
    
    # first entry corresponds to objective function value (number of tasks completed)

    λ_j.C += S[i,j]
    
    # uptading unreachable vertices from j now  
    
    for k in 1:n
        
        if λ_j.V[k] == 0

            # We take into account the time a vehicle would have to wait to complete two consecutive tasks
            
            if λ_j.R[1] + T[j,k] > W[k,2] || λ_j.R[1] + T[j,k] < W[k,1] - 5 || (λ_j.R[1] + T[j,k]) > L # Here, 5 is the limit imposed for the idle time of the vehicle

                λ_j.V[k] = Inf
                
            end

        end
        
    end
    
    λ_j.s = sum(λ_j.V .> 0) # measuring unreachable vertices (visited, impossible to visit or not so great to visit)  
        
    return λ_j
    
end

Extend

## EFF

In [11]:
"""
Functionality: maintain the Lambda sets totally incomparable. Verifies which label can enter and apply the domination rule to remove
labels from the set.

### Input:
    Lambda - set of labels of a vertice;
    F - set of newly extended labels (that will possibly enter Lambda);
    bests - best objective function value (the entire label) obtained so far.

### Output:
    tmplambda - set Lambda possibly altered;
    changed - boolean that stores if Lambda was altered;
    bests - best objective function value (the entire label) obtained so far.
"""
function EFF(Λ :: Set, F :: Set, bests)
        
    dominated = false
    
    changed = false
    
    tmplambda = copy(Λ)
    
    if isempty(Λ) && !isempty(F)
        
        changed = true 
        
        for λ in F
            if λ.C < bests.C
                bests = λ
            end
        end
        
        return F, changed, bests
        
    else
        
        if isempty(F)
            
            changed = false
            
            return Λ, changed, bests
            
        else
            
            for λ_f in F
                
                for λ in tmplambda 

                    if λ <= λ_f && λ != λ_f
                        
                        dominated = true
                        
                        println("Dominated $(λ_f) by $(λ)")
                        
                    end
                    
                    
                    if (λ == λ_f) || dominated
                        
                        break
                        
                    end
                    
                    
                end
                
                if !dominated && !(λ_f in tmplambda)
                    
                    changed = true
                                        
                    for λ in tmplambda

                        if λ_f <= λ
                            
                            dominated = true 
                            
                            println("Dominated $(λ) by $(λ_f)")
                            
                            if dominated
                            
                                tmplambda = setdiff(tmplambda, Set([λ]))
                                
                            end
                            
                        end
                        
                    end
                    
                    tmplambda = union!(tmplambda, Set([λ_f]))
                                        
                    if λ_f.C < bests.C
                        
                        bests = λ_f
                    
                    end
                    
                end
                
                dominated = false
            
            end
            
        end
        
    end
    
    return tmplambda, changed, bests
    
end

EFF

## Data creation

In [12]:
# Creating data for the small instance

# distance matrix C (between sites of delivery)

m = 3 # number of sites

r = 3 # number of original tasks (not containing artificial ones)

n = r + 2

C = Array{Float64}(undef, m, m)

for i in 1:m

    C[i,i] = 0.0

end

C[1,2] = 3 

C[1,3] = 5

C[2,3] = 10

for i in 0:m-1

    for j in 0:m-1

        C[m-i,m-j] = C[m-j,m-i]

    end

end

task = Array{Int64}(undef, r, 2)

task[1,:] = [1, 2]

task[2,:] = [1, 3]

task[3,:] = [2, 3]

L = 40.0

W = Array{Float64}(undef, n, 2)

W[1,:] = [0, L]

W[2,:] = [3.0, 7]

W[3,:] = [0, 5]

W[4,:] = [0, 13]

W[5,:] = [0, L]



2-element Vector{Float64}:
  0.0
 40.0

In [15]:
"""
Functionality: creates data according parameters for the ESPPRC.

### Input:
    n - number of tasks(certices) of the graph (including origin and destination);
    L - Float64 upper limit for time resource;
    dual - matrix 4x1 including dual variables (in order: lambda_0, lambda_1, lambda_2 AND lambda_3)
    C - matrix mxm corresponding to the distance matrix between the m cities;
    task - matrix(n-2)x2 where first column corresponds to origin and second column destination of the task represented by that ith row
    W - matrix nx2 with interval of execution of each task

### Output:
    V, S, T, L, W, Lambda, n - output that will be the according input of ESPPRC (check ESPPRC documentation)

"""
function data(n, L, dual, C, task, W)

    V = Set([i for i in 1:n])

    
    T = zeros(n,n)

    for i in 1:n

        T[i,i] = 100

    end

    T[2:n-1,1] = [100.0, 100, 100]

    T[n,:] = [100, 100, 100, 100, 100]

    T[1:n-1,n] = [0, 0, 0, 0]

    for i in 2:n-1

        for j in 2:n-1 

            if i!=j

                T[i,j] = C[task[i-1,2], task[j-1,1]] + C[task[j-1,1], task[j-1,2]]

            end

        end

    end


    for j in 2:n-1 

        T[1,j] = C[task[1,1], task[j-1,1]] + C[task[j-1,1], task[j-1,2]]

    end


    Lambda = Array{Any,1}(undef, n)

    label_origin = Label(0, [0.0], zeros(n))

    label_origin.V[1] = 1.0

    Lambda[1] = Set([label_origin])

    for i in 2:n
        Lambda[i] = Set([])
    end

    S = Array{Float64}(undef, n, n)

    S[:, 1] .= 0

    S[:, end] .= 0

    S[1, end] = 0

    for i in 2:n

        for j in 2:n-1

            S[i,j] = -1 - dual[j]

        end

    end
    
    for j in 2:n-1

        S[1,j] = -1 - dual[j] - dual[1]

    end

    return V, S, T, L, W, Lambda, n
end

data

## Examples

In [16]:
dual = [0, 0, 0, -1]

V, S, T, L, W, Lambda, n = data(5, 40, dual, C, task, W)
ESPPRC(V, S, T, L, W, Lambda, n)

Dominated Label(0.0, [13.0], 4, [1.0, Inf, Inf, 2.0, 0.0]) by Label(-1.0, [13.0], 4, [1.0, 2.0, Inf, 3.0, 0.0])
Dominated Label(-1.0, [5.0], 5, [1.0, Inf, 2.0, Inf, 3.0]) by Label(-1.0, [3.0], 5, [1.0, 2.0, Inf, Inf, 3.0])
Dominated Label(-1.0, [13.0], 5, [1.0, 2.0, Inf, 3.0, 4.0]) by Label(-1.0, [3.0], 5, [1.0, 2.0, Inf, Inf, 3.0])


(Any[Set(Label[Label(0.0, [0.0], 0, [1.0, 0.0, 0.0, 0.0, 0.0])]), Set(Any[Label(-1.0, [3.0], 3, [1.0, 2.0, Inf, 0.0, 0.0])]), Set(Any[Label(-1.0, [5.0], 4, [1.0, Inf, 2.0, Inf, 0.0])]), Set(Any[Label(-1.0, [13.0], 4, [1.0, 2.0, Inf, 3.0, 0.0])]), Set(Any[Label(0.0, [0.0], 5, [1.0, Inf, Inf, Inf, 2.0]), Label(-1.0, [3.0], 5, [1.0, 2.0, Inf, Inf, 3.0])])], Label(-1.0, [3.0], 3, [1.0, 2.0, Inf, 0.0, 0.0]))